# Remove unnecessary data columns

In [9]:
from pyspark import SQLContext
from time import time

In [24]:
import pyspark
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession\
        .builder\
        .appName("Project_DE11_test")\
        .master("spark://host-192-168-2-13-de1:7077")\
        .config("spark.cores.max", 4)\
        .config("spark.executor.memory", "6G")\
        .config("spark.executor.cores", 4)\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", False)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout", "30s")\
        .config("spark.driver.port", 10000)\
        .config("spark.blockManager.port", 10006)\
        .getOrCreate()

spark_context = spark_session.sparkContext
#sql_context = SQLContext(spark_context)# read the json file into dataframe
# spark_session = SparkSession\
#         .builder\
#         .master("spark://host-192-168-2-20-de1:7077") \
#         .appName("Project_DE11")\
#         .config("spark.dynamicAllocation.enabled", True)\
#         .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
#         .config("spark.shuffle.service.enabled", False)\
#         .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
#         .config("spark.executor.cores",2)\
#         .config("spark.driver.port",9999)\
#         .config("spark.blockManager.port",10005)\
#         .getOrCreate()

In [30]:
# release the cores for another application!

spark_context.stop()


# Selection of posts that fit criteria

# Analysis of Data

# Subreddits in which people prefer to write longer posts

In [2]:
# ------ Install the necessary environment ------
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3156 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1175 kB]
Fetched 4445 kB in 2s (1880 kB/s)                         
Reading package lists... Done
Building dependency tree       
Reading state information... Done
21 packages can be upgraded. Run 'apt list --upgradable' to see them.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exitin

In [3]:
import os
import sys
import findspark

# make pyspark importable as a regular library.
findspark.init()
findspark.find()

'/home/ubuntu/spark-3.5.1-bin-hadoop3'

### Initialize pyspark and load the data

In [ ]:
import pyspark
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from google.colab import drive
from pyspark.sql.functions import split, col, udf, avg
from pyspark.sql.types import IntegerType



spark_session = SparkSession \
       .builder \
       .appName("Ankur Roy")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

drive.mount('/content/drive')

df = spark_session.read.json("hdfs://host-192-168-2-13-de1:9000/user/ubuntu/reduced_set.json")

### Data Processing

In [ ]:
line_count = df.count()

print("Number of lines in English transcripts:", line_count)

selected_df = df.select("author", "content", "content_len", "summary", "summary_len", "subreddit")

length_filter_df = selected_df.filter((selected_df["content_len"] > 50) & (selected_df["summary_len"] > 10))

num_long_posts = length_filter_df.count()

print("Number of long posts", num_long_posts)

length_filter_df = length_filter_df.withColumn("summary_words", split(col("summary"), " "))
length_filter_df = length_filter_df.withColumn("content_words", split(col("content"), " "))



#------ Utilize User-Defined Function to compute the number of common words-------
def count_common_words(summary_words, content_words):
    return len(set(summary_words) & set(content_words))

# Register the UDF
count_common_words_udf = udf(count_common_words, IntegerType())

# Count the number of common words
# Common words refer to those words which both appear in the summary and the content of posts.

length_filter_df = length_filter_df.withColumn("num_common_words",
                   count_common_words_udf(col("summary_words"), col("content_words")))

# Show the DataFrame
length_filter_df.show(truncate=False)
average_common_words = length_filter_df.agg(avg("num_common_words")).collect()[0][0]

print("Average number of posts' common words:", average_common_words)



quality_df = length_filter_df.filter(col("num_common_words") >= 10)

print(quality_df.count())

quality_df.show()

average_common_words = quality_df.agg(avg("num_common_words")).collect()[0][0]

print("Average number of common words in posts with more than 10 common words:", average_common_words)



#------ List the top 20 subreddits in which prefer to write longer posts-------

count_df = quality_df.groupBy("subreddit").count()
count_df = count_df.orderBy(col("count").desc())
# Show the DataFrame
count_df.show()